# train_emo_model

In [ ]:
!pip install tensorflow keras numpy opencv-python


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!unzip -q "/content/drive/MyDrive/face_emo.zip" -d /content


In [7]:
%cd /content/face_emo


/content/face_emo


In [8]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Dropout,
    Flatten,
    Dense,
    BatchNormalization,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import itertools
from tensorflow.keras import backend as K
import gc

In [9]:
# --- Paths ---
train_data_path = "/content/face_emo/dataset/fer2013/train"
val_data_path = "/content/face_emo/dataset/fer2013/validation"
models_dir = "/content/drive/MyDrive/face_emo/"
best_model_path = os.path.join(models_dir, "best_emotion_model.h5")


# --- Load train and test (split from training folder) ---
def load_and_split_data(train_path, img_size=(48, 48), test_size=0.2, batch_size=64):
    datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=test_size)

    train_gen = datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size,
        subset="training",
        shuffle=True,
    )

    test_gen = datagen.flow_from_directory(
        train_path,
        target_size=img_size,
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size,
        subset="validation",
        shuffle=True,
    )

    return train_gen, test_gen


# --- Load true validation (unseen) data ---
def load_final_validation_data(val_path, img_size=(48, 48), batch_size=64):
    datagen = ImageDataGenerator(rescale=1.0 / 255)
    val_gen = datagen.flow_from_directory(
        val_path,
        target_size=img_size,
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=False,
    )
    return val_gen

In [10]:
# --- Parameters to Grid Search ---
param_grid = {
    "filters": [32, 64],
    "dense_units": [128, 256],
    "dropout_rate": [0.3, 0.5],
}


# --- Build CNN Model ---
def build_model(
    filters=64,
    dense_units=256,
    dropout_rate=0.5,
    input_shape=(48, 48, 1),
    num_classes=7,
):
    model = Sequential()
    model.add(Conv2D(filters, (3, 3), activation="relu", input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Conv2D(filters * 2, (3, 3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    model.add(Dense(dense_units, activation="relu"))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation="softmax"))

    model.compile(
        loss="categorical_crossentropy",
        optimizer=Adam(learning_rate=0.0001),
        metrics=["accuracy"],
    )
    return model

In [ ]:
# --- Grid Search Training ---
def train_with_grid_search():
    os.makedirs(models_dir, exist_ok=True)

    train_gen, test_gen = load_and_split_data(train_data_path)
    final_val_gen = load_final_validation_data(val_data_path)

    best_val_acc = 0
    best_model = None
    best_params = {}

    for params in itertools.product(*param_grid.values()):
        filters, dense_units, dropout_rate = params

        # Skip the first 4 combinations (already done)
        #if (filters, dense_units, dropout_rate) in [
        #   (32, 128, 0.3),
        #   (32, 128, 0.5),
        #   (32, 256, 0.3),
        #   (32, 256, 0.5),
        #   (64, 128, 0.3),
        #   (64, 128, 0.5),
        #]:
        #   print(
        #       f"Skipping already trained model: filters={filters}, dense_units={dense_units}, dropout_rate={dropout_rate}"
        #   )
        #   continue

        print(
            f"Training model with filters={filters}, dense_units={dense_units}, dropout_rate={dropout_rate}"
        )

        model = build_model(filters, dense_units, dropout_rate)

        temp_model_path = os.path.join(
            models_dir, f"temp_model_{filters}_{dense_units}_{int(dropout_rate*10)}.h5"
        )
        checkpoint = ModelCheckpoint(
            temp_model_path, monitor="val_loss", save_best_only=True, verbose=0
        )

        callbacks = [
            EarlyStopping(
                monitor="val_loss", patience=5, restore_best_weights=True, verbose=1
            ),
            checkpoint,
        ]

        model.fit(
            train_gen,
            validation_data=test_gen,
            epochs=20,
            callbacks=callbacks,
            verbose=1,
        )

        val_loss, val_acc = model.evaluate(final_val_gen, verbose=0)
        print(f"Final validation accuracy: {val_acc:.4f}")

        # Keep best model only
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = model
            best_params = {
                "filters": filters,
                "dense_units": dense_units,
                "dropout_rate": dropout_rate,
            }

            # Save best model
            best_model.save(best_model_path)

        # After model training & evaluation
        K.clear_session()
        gc.collect()

        # Delete temp model file
        if os.path.exists(temp_model_path):
            os.remove(temp_model_path)

    if best_model:
        print(
            f"\nBest model saved as '{best_model_path}' with accuracy: {best_val_acc:.4f}"
        )
        print("Best parameters:", best_params)
    else:
        print("No model trained successfully.")


if __name__ == "__main__":
    train_with_grid_search()


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Skipping already trained model: filters=32, dense_units=128, dropout_rate=0.3
Skipping already trained model: filters=32, dense_units=128, dropout_rate=0.5
Skipping already trained model: filters=32, dense_units=256, dropout_rate=0.3
Skipping already trained model: filters=32, dense_units=256, dropout_rate=0.5
Skipping already trained model: filters=64, dense_units=128, dropout_rate=0.3
Skipping already trained model: filters=64, dense_units=128, dropout_rate=0.5
Training model with filters=64, dense_units=256, dropout_rate=0.3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 817ms/step - accuracy: 0.2517 - loss: 2.1757

359/359 ━━━━━━━━━━━━━━━━━━━━ 313s 861ms/step - accuracy: 0.2518 - loss: 2.1748 - val_accuracy: 0.1425 - val_loss: 4.0955
Epoch 2/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 842ms/step - accuracy: 0.3517 - loss: 1.6516

359/359 ━━━━━━━━━━━━━━━━━━━━ 319s 888ms/step - accuracy: 0.3517 - loss: 1.6516 - val_accuracy: 0.3809 - val_loss: 1.8228
Epoch 3/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 835ms/step - accuracy: 0.3871 - loss: 1.5583

...

359/359 ━━━━━━━━━━━━━━━━━━━━ 308s 859ms/step - accuracy: 0.4252 - loss: 1.4678 - val_accuracy: 0.4372 - val_loss: 1.6131
Epoch 13/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 819ms/step - accuracy: 0.4388 - loss: 1.4339

359/359 ━━━━━━━━━━━━━━━━━━━━ 310s 863ms/step - accuracy: 0.4388 - loss: 1.4339 - val_accuracy: 0.4470 - val_loss: 1.5442
Epoch 14/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 308s 857ms/step - accuracy: 0.4440 - loss: 1.4266 - val_accuracy: 0.4456 - val_loss: 1.5798
Epoch 15/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 310s 863ms/step - accuracy: 0.4614 - loss: 1.3877 - val_accuracy: 0.4531 - val_loss: 1.5941
Epoch 16/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 821ms/step - accuracy: 0.4629 - loss: 1.3832

359/359 ━━━━━━━━━━━━━━━━━━━━ 311s 866ms/step - accuracy: 0.4629 - loss: 1.3832 - val_accuracy: 0.4626 - val_loss: 1.4825
Epoch 17/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 319s 858ms/step - accuracy: 0.4791 - loss: 1.3514 - val_accuracy: 0.4646 - val_loss: 1.5248
Epoch 18/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 809ms/step - accuracy: 0.4860 - loss: 1.3358

359/359 ━━━━━━━━━━━━━━━━━━━━ 307s 854ms/step - accuracy: 0.4859 - loss: 1.3358 - val_accuracy: 0.4701 - val_loss: 1.4805
Epoch 19/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 322s 855ms/step - accuracy: 0.4812 - loss: 1.3312 - val_accuracy: 0.4741 - val_loss: 1.5420
Epoch 20/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 809ms/step - accuracy: 0.4952 - loss: 1.3026

359/359 ━━━━━━━━━━━━━━━━━━━━ 327s 868ms/step - accuracy: 0.4952 - loss: 1.3026 - val_accuracy: 0.4863 - val_loss: 1.4273
Epoch 21/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 307s 857ms/step - accuracy: 0.5036 - loss: 1.2871 - val_accuracy: 0.4764 - val_loss: 1.5713
Epoch 22/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 323s 861ms/step - accuracy: 0.5100 - loss: 1.2711 - val_accuracy: 0.4935 - val_loss: 1.4434
Epoch 23/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 307s 853ms/step - accuracy: 0.5186 - loss: 1.2469 - val_accuracy: 0.4856 - val_loss: 1.5841
Epoch 24/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 327s 868ms/step - accuracy: 0.5228 - loss: 1.2271 - val_accuracy: 0.4956 - val_loss: 1.5602
Epoch 25/25
359/359 ━━━━━━━━━━━━━━━━━━━━ 323s 871ms/step - accuracy: 0.5362 - loss: 1.2134 - val_accuracy: 0.4943 - val_loss: 1.5424
Epoch 25: early stopping
Restoring model weights from the end of the best epoch: 20.
Final validation accuracy: 0.4978

Best model saved as '/content/drive/MyDrive/face_emo/best_emotion_model.h5' with accuracy: 0.5